In [90]:
import data_loader
import pandas as pd
import seaborn as sb
import numpy as np
init_train = data_loader.load_train_data("../data/adult.data", valid_rate=1, is_df = "True")
test = data_loader.load_test_data("../data/adult.test", is_df = "True")
train = init_train[1]

In [91]:
col_map = ["age", "workclass", "final_weight", "education", "education_num", "marital_status", "occupation", "relationship", "race", "sex", "cap_gain", "cap_loss", "hrs_p_week","country","income"]
train.columns = col_map

In [92]:
print(train)

       age          workclass  final_weight      education  education_num  \
24337   62          Local-gov         68268        HS-grad              9   
17049   50            Private        215990        HS-grad              9   
21016   36            Private        185405        HS-grad              9   
2790    64            Private        258006   Some-college             10   
13511   28   Self-emp-not-inc         39388      Assoc-voc             11   
29288   19            Private        100790        HS-grad              9   
2161    21            Private         60639   Some-college             10   
12546   39            Private        113481      Bachelors             13   
20828   57            Private         96779        5th-6th              3   
210     44          Local-gov        160943        HS-grad              9   
20261   66            Private        214469        HS-grad              9   
27729   34            Private        127610      Bachelors             13   

In [93]:
train['income'] = train['income'].map({ " <=50K": 0, " >50K": 1})
Results = np.copy(train['income'].values)
train.drop(columns="income",inplace=True)

In [108]:
for column in train.columns:
    su = (train[column] == ' ?').sum()
    if su > 0:
        print("Column: ",column, " Sum: ",su)

Column:  workclass  Sum:  1836
Column:  occupation  Sum:  1843
Column:  country  Sum:  583


In [114]:
#train.drop(columns=['workclass','occupation','country'], )
new_train = train[(train.country != ' ?' || train.workclass != ' ?' || train.occupation != ' ?')]
for column in new_train.columns:
    su = (new_train[column] == ' ?').sum()
    if su > 0:
        print("Column: ",column, " Sum: ",su)

SyntaxError: invalid syntax (<ipython-input-114-cc18548821e0>, line 2)

In [105]:
print(help(train.drop))

Help on method drop in module pandas.core.frame:

drop(labels=None, axis=0, index=None, columns=None, level=None, inplace=False, errors='raise') method of pandas.core.frame.DataFrame instance
    Drop specified labels from rows or columns.
    
    Remove rows or columns by specifying label names and corresponding
    axis, or by specifying directly index or column names. When using a
    multi-index, labels on different levels can be removed by specifying
    the level.
    
    Parameters
    ----------
    labels : single label or list-like
        Index or column labels to drop.
    axis : {0 or 'index', 1 or 'columns'}, default 0
        Whether to drop labels from the index (0 or 'index') or
        columns (1 or 'columns').
    index, columns : single label or list-like
        Alternative to specifying axis (``labels, axis=1``
        is equivalent to ``columns=labels``).
    
        .. versionadded:: 0.21.0
    level : int or level name, optional
        For MultiIndex, level

In [6]:
print(help(train.filter))

Help on method filter in module pandas.core.generic:

filter(items=None, like=None, regex=None, axis=None) method of pandas.core.frame.DataFrame instance
    Subset rows or columns of dataframe according to labels in
    the specified index.
    
    Note that this routine does not filter a dataframe on its
    contents. The filter is applied to the labels of the index.
    
    Parameters
    ----------
    items : list-like
        List of axis to restrict to (must not all be present).
    like : string
        Keep axis where "arg in col == True".
    regex : string (regular expression)
        Keep axis with re.search(regex, col) == True.
    axis : int or string axis name
        The axis to filter on.  By default this is the info axis,
        'index' for Series, 'columns' for DataFrame.
    
    Returns
    -------
    same type as input object
    
    See Also
    --------
    DataFrame.loc
    
    Notes
    -----
    The ``items``, ``like``, and ``regex`` parameters are
    